##1.Imports

In [0]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 2.Load Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')
dataset=pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/IMDB Dataset.csv")
dataset.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## 3.Data cleaning

In [0]:
def rem_stopwords(row_val):
  corpus=[]
  cleaned=(re.sub("[^a-zA-Z]",' ',row_val))
  cleaned=cleaned.lower()
  cleaned_list=cleaned.split()
  ps=PorterStemmer()
  imp_words=[ps.stem(word) for word in cleaned_list if word not in set(stopwords.words('english'))]
  imp_words = " ".join(imp_words)
  corpus.append(imp_words)
  return corpus

In [0]:
dataset['imp_words']=dataset['review'].apply(rem_stopwords)
corpus=[]
for i in dataset['imp_words']:
  corpus.append(i[0])

## 4. Bag of Word Model

In [0]:
# We have a prefectly balanced dataset
dataset.iloc[:, 1].value_counts()

negative    10
positive    10
Name: sentiment, dtype: int64

In [0]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le = preprocessing.LabelEncoder()
dataset['sentiment'] = le.fit_transform(dataset.sentiment.values)

In [0]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

## 5. Predicitve Models

a. Train-Test Split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

b. Naive Bayes Classifer

In [0]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)
#Predicting
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.72      0.85      0.78      5035
    positive       0.82      0.67      0.74      4965

    accuracy                           0.76     10000
   macro avg       0.77      0.76      0.76     10000
weighted avg       0.77      0.76      0.76     10000



c. Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

    negative       0.88      0.87      0.87      5035
    positive       0.87      0.88      0.87      4965

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



d. Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model=model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.83      0.84      0.84      5035
    positive       0.84      0.83      0.83      4965

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [0]:
text = "The food quality is very very bad had order some soup it was so terrible could eat more than a spoonful. They need to change the chef at the earliest."
corpus2 = []
review2 = re.sub("[^a-zA-z]", ' ', text)
review2 = review2.lower()
review2 = review2.split()
ps2 = PorterStemmer()
review2 = [ps2.stem(word) for word in review2 if not word in set(stopwords.words('english'))]
review2 = " ".join(review2)
corpus2.append(review2)
cv2 = CountVectorizer(max_features = 1500)
X2 = cv2.fit_transform(corpus + corpus2).toarray()
my = X2[-1].reshape(1, -1)
#result = classifier.predict(my)
result = model.predict(my)
if result == 1:
    answer = "Positive"
else:
    answer = "Negative"
    
print(answer)
print(result)

Negative
['negative']


In [0]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
X_train.shape

(40000, 1500)

In [0]:
# create the model
model = Sequential()
model.add(Embedding(5000, 32, input_length=X_train.shape[1]))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1500, 32)          160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 48000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               12000250  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 12,160,501
Trainable params: 12,160,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
 - 108s - loss: 0.2669 - accuracy: 0.8869 - val_loss: 0.3031 - val_accuracy: 0.8740
Epoch 2/5
 - 107s - loss: 0.2425 - accuracy: 0.8967 - val_loss: 0.3107 - val_accuracy: 0.8689
Epoch 3/5
